# Exercise 3: Finite size scaling

In this exercise, we will determine the critical temperature and critical exponents of the
Ising model.

a) First, we need finite size data. Since generating the data takes some time, it is useful
to save it to disk. Generate your own finite size data from last weeks program (which
can take some time to get well converged results), and/or download finite size data
provided on the GitLab page. Inspect generate_data.py to find out how the data
is structured and how to load the data. Plot the specific heat CV and magnetic
susceptibility χ.

Recall: 
$$ C_v = \frac{1}{k_BT^2} \left(\left<E^2\right> - \left<E\right>^2\right)  $$

$$ \chi = \frac{1}{k_BT} \left(\left<M^2\right> - \left<M\right>^2\right) $$


In [ ]:
%matplotlib qt5
from ising_model import IsingModel
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
def calc_Cv(E: np.ndarray, T: float, kb: float = 1) -> float:
    mean_E = np.mean(E)
    mean_E2 = np.mean(E**2)
    return 1/(kb*T**2) * (mean_E2 - mean_E**2)

calc_Cv_vectorized = np.vectorize(calc_Cv, excluded=["E"], signature="(m),()->()")

def calc_chi(M: np.ndarray, kbT: float) -> float:
    mean_M = np.mean(M)
    mean_M2 = np.mean(M**2)
    return 1/(kbT) * (mean_M2 - mean_M**2)

calc_chi_vectorized = np.vectorize(calc_chi, excluded=["M"], signature="(m),()->()")


In [ ]:
def generate_data(L: int, T0: float = 1, Tn: float = 3.5, N: int = 100) -> tuple[np.ndarray, np.ndarray]:
    system = IsingModel(1.0, L, L)
    Ts = np.linspace(T0, Tn, N)

    # do some thermalization
    for _ in range(100):
        system.iterate_swendsen_wang(Ts[0])

    return system.sweep_swendsen_wang(Ts, 1000)
Ts = np.linspace(1, 3.5, 100)

In [ ]:
Es_20, Ms_20 = generate_data(20)
# Es_100, Ms_100 = generate_data(100)

In [ ]:
Cvs_20 = calc_Cv_vectorized(Es_20, Ts)
chis_20 = calc_chi_vectorized(Ms_20, Ts)
Cvs_100 = calc_Cv_vectorized(Es_100, Ts)
chis_100 = calc_chi_vectorized(Ms_100, Ts)

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.plot(Ts, Cvs_20, label="L=20")
plt.plot(Ts, Cvs_100, label="L=100")
plt.xlabel("T")
plt.legend()
plt.title("Cv")
plt.subplot(1,2,2)
plt.plot(Ts, chis_20, label="L=20")
plt.plot(Ts, chis_100, label="L=100")
plt.xlabel("T")
plt.title("$\\chi$")
plt.legend()
plt.show()